Вопросы для повторения:
* Что такое стэк и куча?
* Что такое указатель?
* Чему равно `p`?

    ```
    std::uint64_t *p = 1000;
    p = p + 4;
    ```

* Где хранятся данные `"hello world"` при исполнении программы?
* В чём разница в вводе-выводе в стиле С и С++?
* В чём идея forward declaration и для чего используется?

<br />

### Лекция 2. Пространства имён, функции и ввод-вывод

<br />

##### Функции, объявление и определение (declaration / definition)

Определение (definition) функции - описание её "интерфейса" (сигнатуры, возвращаемого типа и квалификаторов) И реализации.

```c++
float abs(float x)
{
    if (x >= 0)
        return x;
    return -x;
}

[[nodiscard]] float min_value(const std::vector<float>& items) noexcept
{
    float rv = items.front();
    for (const float item : items)
        rv = std::min(rv, item);
    return rv;
}
```

Замечание:
* для поиска минимума, конечно же, используйте `std::min_element`
* а для вычисления абсолютных величин функцию `abs` из `#include <cmath>`

Объявление (declaration) функции - описание её "интерфейса"

```c++
float abs(float x);

[[nodiscard]] float min_value(const std::vector<float>& items) noexcept;
```

Если не прописано явно, у функции в программе может быть несколько объявлений, но не больше одного определения.

<br />

##### Передача аргументов в функцию и возвращаемое значение

Передача по значению и по ссылке
(нарисовать происходящее в памяти)

```c++
std::vector<float> v = {1.f, 2.f, 3.f};
```

Передача по значению - создание копии аргумента

```c++
float min_value_1(std::vector<float> items);

float m = min_value_1(v);
```

Передача по ссылке - работа с аргументом

```c++
float min_value_2(std::vector<float>& items);

float m = min_value_2(v);
```

Запрет на модификацию

```c++
float min_value_3(const std::vector<float>& items);

float m = min_value_3(v);
```

Что здесь происходит?

```c++
float f(std::vector<float>* items);
float f(const std::vector<float>* items);
float f(std::vector<float>* const items);
```

<br />

Возращаемое значение

```c++
void say_hello(const std::string& name)
{
    std::cout << "hello, " << name;
}
```

```c++
std::vector<std::string> make_team()
{
    return { "Bifur", "Bofur", "Bombur", "Oin",
             "Gloin", "Fili", "Nori", "Dori",
             "Dwalin", "Ori", "Balin", "Kili" };
}
```

```c++
bool append_teamlead(std::vector<std::string>& team)
{
    team.push_back("Thorin");
    return true;
}
```

Немного примеров посложнее

```c++
std::string* find_balin(const std::vector<std::string>& team)
{
    for (const auto& member : team)
        if (member == "Balin")
            return &member;
    return nullptr;
}

// usage 1
std::vector<std::string> team = make_team();
if (std::string* maybe_balin = find_balin(team))
    std::cout << *maybe_balin;

// usage 2
if (std::string* maybe_balin = find_balin({"Ori", "Gloin", "Balin"}))
    std::cout << *maybe_balin;  // OOOPS
```

Ещё пример:

```c++
const std::string& get_value_from_cache(Cache c, int key, const std::string& default_value)
{
    if (c.contains(key))
        return c[key];
    return default_value;
}

// usage 1
const std::string default_value = "Bilbo";
std::cout << get_value_from_cache(cache, missing_key, default_value);  // OK
        
// usage 2
std::cout << get_value_from_cache(cache, missing_key, "Bilbo");  // OK
        
// usage 3
std::string value = get_value_from_cache(cache, missing_key, "Bilbo");  // OK
std::cout << value;
        
// usage 4
const std::string& value = get_value_from_cache(cache, missing_key, "Bilbo");
std::cout << value;  // OOOOPS
```

Показать пример на godbolt.org на clang 8.0.0 с разными оптимизациями, чтобы наглядно продемонстрировать ub

```c++
#include <iostream>
#include <string>

const std::string& f(const bool x,
                     const std::string& default_value1,
                     const std::string& default_value2)
{
    return x ? default_value1 : default_value2;
}

int main()
{
    std::cout << "hello world" << std::endl;

    const std::string& s = f(true, "123", "12345");
    std::cout << s << std::endl;
    return 0;
}
```

<br />

##### Порядок вычисления аргументов

__Вопрос__: что будет распечатано в примерах?

Пример 1:

```c++
int a() { std::cout << 'a'; return 1; }
int b() { std::cout << 'b'; return 2; }
int c() { std::cout << 'c'; return 3; }

int sum(int a, int b int c) {
    return a + b + c;
}

int main() {
    std::cout << sum(a(), b(), c()) << std::endl;
    return 0;
}
```

Пример 2:

```c++
int a() { std::cout << 'a'; return 1; }
int b() { std::cout << 'b'; return 2; }
int c() { std::cout << 'c'; return 3; }

int main() {
    std::cout << a() + b() * c() << std::endl;
    return 0;
}
```

Ответ:

__Порядок вычисления аргументов не определён!__

https://en.cppreference.com/w/cpp/language/eval_order

<br />

##### Значения аргументов по умолчанию

Можно задавать значения аргументов по умолчанию:

```c++
std::string convert_to_string(int value, int base = 10);
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n");
```

Но такие аргументы должны быть последними:

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines);  // compilation ERROR!
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines = false);  // OK
```

<br />

##### Перегрузка функции

https://en.cppreference.com/book/intro/function_overloading

Задача - реализовать конвертацию всего в строку. Желательно единообразно и чтобы: есть способ - компилируется и работает, нет способа - не компилируется.

```c++
std::string convert_to_string(int x);       // 1
std::string convert_to_string(unsigned x);  // 2
std::string convert_to_string(float x);     // 3

std::cout << convert_to_string(5);    // 1
std::cout << convert_to_string(5u);   // 2
std::cout << convert_to_string(5.f);  // 3
```

<br />

##### Пространства имён и name mangling при компиляции

https://en.wikipedia.org/wiki/Name_mangling

Пространства времён решают проблему, когда функция с одинаковым именем имеет две реализации в разных билиотеках

```c++
// library for json parsing

// return version of json parsing library
std::string get_version();
```

```c++
// library for image manipulation

// return version of image manipulation library
std::string get_version();
```

Такой код скомпилируется, но не слинкуется, т.к. для функции `get_version` нарушено ODR-правило (One Definition Rule)

```c++
// library for json parsing

namespace cool_json_parser
{
    // return version of json parsing library
    std::string get_version();
}
```

```c++
// library for image manipulation

namespace cool_image_processing
{
   // return version of image manipulation library
    std::string get_version(); 
}
```

Такой код слинкуется, т.к. работает механизм name mangling во время компиляции

Показать пример на godbolt.org, не забыть убрать галочку demangle

```c++
#include <iostream>
#include <string>

std::string get_version(int x, int y, long z);

namespace cool_json_parser
{
    std::string get_version(int x, int y, long z);
}

namespace cool_image_processing
{
    std::string get_version(int x, int y, long z);
}

int main()
{
    std::cout << get_version(0, 0, 0);
    std::cout << cool_json_parser::get_version(0, 0, 0);
    std::cout << cool_image_processing::get_version(0, 0, 0);
    return 0;
}
```

В результате name mangling компилятор clang создаёт символы:
* `_Z11get_versionB5cxx11iil`
* `_ZN16cool_json_parser11get_versionB5cxx11Eiil`
* `_ZN21cool_image_processing11get_versionB5cxx11Eiil`

Какой из них к чему относится, догадаться не сложно. 

<br />

##### ADL (Argument Dependent Lookup)

https://en.cppreference.com/w/cpp/language/adl

Теперь мы реализовали свой класс `Point` в нашем `namespace math` и хотим, чтобы его объекты тоже можно было конвертировать в строку. Желательно единообразно.

```c++
namespace mymath
{
    struct Point
    {
        float x;
        float y;
    }
    
    std::string convert_to_string(Point p)
    {
        return convert_to_string(p.x) + " " + convert_to_string(p.y);
    }
}

int main()
{
    mymath::Point p{3.f, 4.f};
    
    // явно не указано пространство имён mymath у функции
    // convert_to_string, но ADL нужную функцию найдёт 
    std::cout << convert_to_string(p) << std::endl;
    
    return 0;
}
```

ADL - набор правил, который добавляет к области поиска функции пространства имён, содержащих её аргументы

<br />

##### Функции (операторы) ввода-вывода

Операторы - те же функции, но в альтернативной записи:

```c++
namespace std
{
    // есть в стандартной библиотеке
    string operator + (const string& l, const string& r);
}

namespace mymath
{
    // личный оператор сложения для точек
    Point operator + (const Point& l, const Point& r);
}

int main()
{
    std::string s1 = "s1", s2 = "s2";
    mymath::Point p1{0, 1}, p2{1, 0};
    
    std::string s = s1 + s2;
    mymath::Point p = p1 + p2;
    
    return 0;
}
```

Можно доопределить свои операторы для стандартных типов

```c++
namespace std
{
    string operator * (const string& l, unsigned count)
    {
        string rv;
        for (unsigned i = 0; i < count; ++i)
            rv += l;
        return rv;
    }
}

std::cout << std::string("nom ") * 3;  // "nom nom nom "
```

И их использование строится на тех же правилах: перегрузка + ADL

Пример для операторов вывода:

```c++
// from standard lib
namespace std
{
    std::ostream& operator << (std::ostream& os, int x);
    std::ostream& operator << (std::ostream& os, float x);
    std::ostream& operator << (std::ostream& os, const char* x);
    std::ostream& operator << (std::ostream& os, const std::string& x);
}

// your own
namespace mymath
{
    std::ostream& operator << (std::ostream& os, Point p)
    {
        return os << p.x << " " << p.y;
    }
}
```

<br />

##### nodiscard && maybe_unused

`[[maybe_unused]]` - подсказка компилятору, что сущность может быть неиспользуемой, и это нормально. Подавляет warning-и о таких сущностях.

https://en.cppreference.com/w/cpp/language/attributes/maybe_unused

Пример из документации:

```c++
[[maybe_unused]] void f([[maybe_unused]] bool thing1,
                        [[maybe_unused]] bool thing2)
{
   [[maybe_unused]] bool b = thing1 && thing2;
   assert(b); // in release mode, assert is compiled out, and b is unused
              // no warning because it is declared [[maybe_unused]]
} // parameters thing1 and thing2 are not used, no warning
```

Пример 1:

Зависимость от режима компиляции (debug/release)

```c++
void draw_3_heroes_picture(const std::vector<People>& team)
{
    {
        [[maybe_unused]] People* ilya = find_member(team, "Ilya");
        assert(ilya);  // assert is noop in release mode
        
        [[maybe_unused]] People* dobryna = find_member(team, "Dobryna");
        assert(dobryna);  // assert is noop in release mode
        
        [[maybe_unused]] People* alesha = find_member(team, "Alesha");
        assert(alesha);  // assert is noop in release mode
    }
    ... 
}
```

Пример 2:

Зависимость от флагов компиляции

```c++
#ifdef ENABLE_LOGGING
    #define LOG(x) std::clog << x << std::endl
#else
    #define LOG(x)
#endif

// return erased elements count
int erase_element_inplace(std::vector<int>& vec, int value);


[[maybe_unused]] const int erased_fives = erase_element_inplace(vec, 5);
LOG("erased elements count");
LOG(x);
```

<br />

`[[nodiscard]]` - запрет программисту игнорировать возвращаемое значение функции.

```c++
[[nodiscard]] int sqr(int x) {
    return x * x;
}
```

Использование:

```c++
int nine = sqr(3); // OK

sqr(2); // compilation error
```

<br />

Рекомендации по использованию:
1. _Рекомендаций от сообщества нет_
2. Наверное, не стоит все подряд не-`void` функции помечать `[[nodiscard]]`, но это не точно
3. Стоит помечать функции `[[nodiscard]]`, если по имени не очевидно предназначение. Примеры:

```c++
class set
{
    ...

    // return the new set having this with |value| added
    [[nodiscard]] set add(int value);
};

set s = {1, 2, 3};
s.add(4);  // compilation error: incorrect usage
```

```c++
// класс динамического массива из стандартной библиотеки
class vector
{
    ...
    // пуст ли массив
    [[nodiscard]] bool empty();
};

// класс динамического массива из Unreal Engine 4
class TArray
{
    ...
    // очистить
    void Empty();
};
```

__Замечание__: `std::vector::empty` помечен `[[nodiscard]]` начиная с С++20

__Вопрос__:
Логика работает так: по умолчанию функция `discardable`, но её можно пометить `[[nodiscard]]`. Но (за редким исключением) если функция возвращает не-`void`, значение не должно быть проигнорировано. Логичнее было бы сделать наоборот: ввести атрибут `[[discardable]]`, а по умолчанию сделать `nodiscard`. Меньше кода, меньше ошибок. Так не сделали. Почему?